# Loading data

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install nibabel

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob

import nibabel as nib
import cv2
import imageio
from tqdm.notebook import tqdm
from ipywidgets import *
from PIL import Image

In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html > /dev/null
!pip install --upgrade kornia > /dev/null
!pip install allennlp==1.1.0.rc4 > /dev/null

In [ ]:
!pip install --upgrade fastai > /dev/null
import fastai; fastai.__version__

In [ ]:
from fastai.basics import *
from fastai.vision.all import *
from fastai.data.transforms import *

In [ ]:
# Creat a meta file

file_list = []
for dirname, _, filenames in os.walk('../input/liver-tumor-segmentation'):
  for filename in filenames:
    #print(os.path.join(dirname, filename))
    file_list.append((dirname, filename))
    
for dirname, _, filenames in os.walk('../input/liver-tumor-segmentation-part-2'):
    for filename in filenames:
        file_list.append((dirname,filename))

df_files = pd.DataFrame(file_list, columns= ['dirname', 'filename'])
df_files.sort_values(by= ['filename'], ascending= True)

# Label

In [ ]:
# Map CT scan and label

df_files["mask_dirname"]= ""; df_files["mask_filename"]= ""
for i in range(131):
    ct = f"volume-{i}.nii"
    mask = f"segmentation-{i}.nii"
    
    df_files.loc[df_files['filename'] == ct, 'mask_filename'] = mask
    df_files.loc[df_files['filename'] == ct, 'mask_dirname'] = "../input/liver-tumor-segmentation/segmentations"

df_files_test= df_files[df_files.mask_filename=='']
# drop segment rows
df_files = df_files[df_files.mask_filename != ''].sort_values(by=['filename']).reset_index(drop=True) 
print(len(df_files))
print(df_files)

In [ ]:
# Reads .nii file and returns pixel array
import nibabel as nib
def read_nii(filepath):
  ct_scan = nib.load(filepath)
  array = ct_scan.get_fdata()
  array = np.rot90(np.array(array))
  return array

In [ ]:
# Read sample
sample = 3
sample_ct = read_nii(df_files.loc[sample,'dirname']+"/"+df_files.loc[sample,'filename'])
sample_mask  = read_nii(df_files.loc[sample,'mask_dirname']+"/"+df_files.loc[sample,'mask_filename'])
print(sample_ct.shape)
print(sample_mask.shape)
print(df_files.loc[sample,'dirname']+"/"+df_files.loc[sample,'filename'])

In [ ]:
print(np.amin(sample_ct), np.amax(sample_ct))
print(np.amin(sample_mask), np.amax(sample_mask))

In [ ]:
# Preprocess the nii file

dicom_windows = types.SimpleNamespace(
    brain = (80, 40),
    subdural = (264, 100),
    stroke = (8, 32),
    brain_bone = (2800, 600),
    brain_sofr = (375, 40),
    lungs = (1500, -600),
    mediastinum = (350, 50),
    abdomen_soft = (400, 50),
    liver = (150, 30),
    spine_soft = (250, 50),
    spine_bone = (1800, 400),
    custom = (200, 60)
)

@patch
def windowed(self:Tensor, w, l):
  px = self.clone()
  px_min = l- w//2
  px_max = 1+ w//2
  px[px<px_min] = px_min
  px[px>px_max] = px_max
  return (px- px_min) / (px_max- px_min)


In [ ]:
plt.imshow(tensor(sample_ct[...,50].astype(np.float32)).windowed(*dicom_windows.liver), cmap=plt.cm.bone);

In [ ]:
# Plots and a slice with all available annotations
def plot_sample(array_list, color_map = 'nipy_spectral'):
  fig = plt.figure(figsize= (18, 15))

  plt.subplot(1, 4, 1)
  plt.imshow(array_list[0], cmap= 'bone')
  plt.title('Original Image')

  plt.subplot(1, 4, 2)
  plt.imshow(tensor(array_list[0].astype(np.float32)).windowed(*dicom_windows.liver), cmap='bone');
  plt.title('Windowed Image')

  plt.subplot(1,4,3)
  plt.imshow(array_list[1], alpha=0.5, cmap=color_map)
  plt.title('Mask')
    
  plt.subplot(1,4,4)
  plt.imshow(array_list[0], cmap='bone')
  plt.imshow(array_list[1], alpha=0.5, cmap=color_map)
  plt.title('Liver & Mask')

  plt.show()

In [ ]:
sample = 421
sampe_slice = tensor(sample_ct[...,sample].astype(np.float32))
plot_sample([sample_ct[...,sample], sample_mask[...,sample]])

In [ ]:
# Cehck the mask valuse
mask = Image.fromarray(sample_mask[...,sample].astype('uint8'), mode="L")
print(mask.shape)
unique, counts = np.unique(mask, return_counts=True)
print( np.array((unique, counts)).T)
plt.imshow(mask , cmap = 'bone')

In [ ]:
# Preprocessing functions
class TensorCTScan(TensorImageBW): _show_args = {'cmap':'bone'}

@patch
def freqhist_bins(self:Tensor, n_bins=100):
    "A function to split the range of pixel values into groups, such that each group has around the same number of pixels"
    imsd = self.view(-1).sort()[0]
    t = torch.cat([tensor([0.001]),
                   torch.arange(n_bins).float()/n_bins+(1/2/n_bins),
                   tensor([0.999])])
    t = (len(imsd)*t).long()
    return imsd[t].unique()
    
@patch
def hist_scaled(self:Tensor, brks=None):
    "Scales a tensor using `freqhist_bins` to values between 0 and 1"
    if self.device.type=='cuda': return self.hist_scaled_pt(brks)
    if brks is None: brks = self.freqhist_bins()
    ys = np.linspace(0., 1., len(brks))
    x = self.numpy().flatten()
    x = np.interp(x, brks.numpy(), ys)
    return tensor(x).reshape(self.shape).clamp(0.,1.)
    
    
@patch
def to_nchan(x:Tensor, wins, bins=None):
    res = [x.windowed(*win) for win in wins]
    if not isinstance(bins,int) or bins!=0: res.append(x.hist_scaled(bins).clamp(0,1))
    dim = [0,1][x.dim()==3]
    return TensorCTScan(torch.stack(res, dim=dim))

@patch
def save_jpg(x:(Tensor), path, wins, bins=None, quality=90):
    fn = Path(path).with_suffix('.jpg')
    x = (x.to_nchan(wins, bins)*255).byte()
    im = Image.fromarray(x.permute(1,2,0).numpy(), mode=['RGB','CMYK'][x.shape[0]==4])
    im.save(fn, quality=quality)

# Unet

In [ ]:
import torch.nn as nn
import torch
from torch import autograd
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, models, transforms

In [ ]:
class Block(nn.Module):
  def __init__(self, in_ch, out_ch):
    super().__init__()
    self.conv1 = nn.Conv2d(in_ch, out_ch, 3)
    self.relu = nn.ReLU()
    self.conv2 = nn.Conv2d(out_ch, out_ch, 3)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.relu(x)
    return x

In [ ]:
enc_block = Block(1, 64)
x = torch.rand(1, 1, 572, 572)
enc_block(x).shape

In [ ]:
class Encoder(nn.Module):
  def __init__(self, chs= (3, 64, 128, 256, 512, 1024)):
    super().__init__()
    self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
    self.pool = nn.MaxPool2d(2)

  def forward(self, x):
    ftrs = []
    for block in self.enc_blocks:
      x = block(x)
      ftrs.append(x)
      x = self.pool(x)
    return ftrs

In [ ]:
encoder = Encoder()
x = torch.randn(1, 3, 572, 572)
ftrs = encoder(x)
for ftr in ftrs:
  print(ftr.shape)

In [ ]:
class Decoder(nn.Module):
  def __init__(self, chs= (1024,  512, 256, 128, 64)):
    super().__init__()
    self.chs = chs
    self.upconvs = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
    self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])

  def forward(self, x, encoder_features):
    for i in range(len(self.chs)-1):
      x = self.upconvs[i](x)
      enc_ftrs = self.crop(encoder_features[i], x)
      x = torch.cat([x, enc_ftrs], dim= 1)
      x = self.dec_blocks[i](x)
    return x
  
  def crop(self, enc_ftrs, x):
    _, _, H, W = x.shape
    enc_ftrs = transforms.CenterCrop([H, W])(enc_ftrs)
    return enc_ftrs

In [ ]:
decoder = Decoder()
x = torch.randn(1, 1024, 28, 28)
decoder(x, ftrs[::-1][1:]).shape

In [ ]:
class UNet(nn.Module):
  def __init__(self, enc_chs= (3, 64, 128, 256, 512, 1024),
              dec_chs= (1024, 512, 256, 128, 64), num_class= 1,
              retain_dim= False, out_sz= (572, 572)):
    super().__init__()
    self.encoder = Encoder(enc_chs)
    self.decoder = Decoder(dec_chs)
    self.head = nn.Conv2d(dec_chs[-1], num_class, 1)
    self.retain_dim = retain_dim
  
  def forward(self, x):
    enc_ftrs = self.encoder(x)
    out = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
    out = self.head(out)
    if self.retain_dim:
      out = F.interpolate(out, out_sz)
    return out

In [ ]:
unet = UNet()
x = torch.randn(1, 3, 572, 572)
unet(x).shape

# Training Process

## Loss function
Using Dice loss as loss function

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()
    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

## Accuracy
using dice coefficient as accuracy

In [ ]:
def dice_coeff(pred, target):
    smooth = 1.
    num = pred.size(0)
    m1 = pred.view(num, -1).float()  # Flatten
    m2 = target.view(num, -1).float()  # Flatten
    intersection = (m1 * m2).sum().float()

    return (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)

## Hyperparameters

In [ ]:
device = 'cuda'
batch_size = 32
learning_rate = 0.0001

In [ ]:
gmodel = UNet()
#print(gmodel)
summary(gmodel, (3, 572, 572))
gmodel= gmodel.to(device)
loss_f = DcieLoss()
optimizer = optim.AdamW(gmodel.parameters(), lr = learning_rate)

In [ ]:
import numpy as np
trainloss = []
validloss = []
trainaccu = []
validaccu = []

In [ ]:
'''import numpy as np
import csv
import pandas as pd
Data = pd.read_csv('./model/output.csv', delimiter= ',', encoding= 'utf-8', header= None)
data = Data.to_numpy()
for i in range(0, np.size(data, axis= 1)):
  trainloss.append(data[0][i])
  validloss.append(data[1][i])
for i in range(0, np.size(data, axis= 1)):
  trainaccu.append(data[2][i])
  validaccu.append(data[3][i])'''


In [ ]:
from tqdm import  tqdm
def train(epoch, model):
    '''if epoch == 0:
      checkpoint=torch.load("./model/checkpoint.ckpt",map_location=device)
      model_state, optimizer_state = checkpoint["model"], checkpoint["optimizer"]
      model.load_state_dict(model_state)
      optimizer.load_state_dict(optimizer_state)'''
    model.train()
    correct = 0
    train_loss = 0
    # for batch_idx, (data, target) in enumerate(train_loader):
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        # data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_f(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        correct += dice_coeff(output, target).sum().item()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx *len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss /= len(train_loader.dataset)
    print('\nTraining  set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    train_loss /= len(train_loader.dataset)
    trainloss.append(train_loss)
    trainaccu.append(correct / len(train_loader.dataset))

In [ ]:
valid_output = []
def valid(model):
    model.eval()
    valid_loss = 0
    correct = 0
    loss = []
    global valid_output
    valid_output = []
    for data, target in tqdm(valid_loader):
        # data, target = Variable(data, volatile = True), Variable(target)
        data, target = data.to(device), target.to(device)
        output = model(data)
        valid_output.append(output.detach().cpu().numpy())
        # Sum up vatch loss
        valid_loss += loss_f(output, target).data.item()
        correct += dice_coeff(output, target).sum().item()
    valid_loss /= len(valid_loader.dataset)
    print('Validation  set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        valid_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))
    validloss.append(valid_loss)
    validaccu.append(correct / len(valid_loader.dataset))
    valid_output = np.concatenate(valid_output)

In [ ]:
import numpy as np
num_iter= 2
for epoch in range(0, num_iter):
    train(epoch, gmodel)
    i = len(validloss)
    valid(gmodel)
    if validloss[i] > validloss[i-1]:
        print("Validation Loss increased")
        break
torch.save({"model": gmodel.state_dict(), "optimizer": optimizer.state_dict()}, "./model/checkpoint.ckpt")

# Result
## Loss and Accuracy of training data and validation data

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(0, len(trainloss)), trainloss, color = 'blue', label = 'training loss')
plt.plot(range(0, len(validloss)), validloss, color = 'red', label = 'validation loss')
plt.legend()
plt.show()

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(range(0, len(trainaccu)), trainaccu, color = 'blue', label = 'training accuracy')
plt.plot(range(0, len(validaccu)), validaccu, color = 'red', label = 'validation accuracy')
plt.legend()
plt.show()

In [ ]:
import csv
with open('./output.csv','w',newline='') as f:
  w = csv.writer(f)
  w.writerow(trainloss)
  w.writerow(validloss)
  w.writerow(trainloss)
  w.writerow(validloss)

Print an sample to compare the predicted result vs. the mask

In [ ]:
sample = 420
sampe_slice = tensor(sample_ct[...,sample].astype(np.float32))
plot_sample([sample_ct[...,sample], valid_output[...,sample]])
plot_sample([sample_ct[...,sample], sample_mask[...,sample]])